In [73]:
#import classes
from random import shuffle
import gensim
import os
import collections
import smart_open
import random
import re
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [74]:
def read_labels(fname, nums=False):
    arr = []
    with smart_open.smart_open(fname) as f:
        for line in enumerate(f):
            s = line[1].decode('utf-8')
            if nums:
                yield line[1].decode('utf-8')
            else:
                if "U" in s:
                    yield 3
                elif "Q" in s:
                    yield 2
                elif "N" in s:
                    yield 1
                else:
                    yield 0

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    sns.heatmap(cm)

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [75]:
train_nums = list(read_labels("Records_Train_Labels.txt"))
test_nums = list(read_labels("Records_Test_Labels.txt"))
train_labels = list(read_labels("Records_Train_Labels.txt", nums=True))
test_labels = list(read_labels("Records_Test_Labels.txt", nums=True))
corpus = []

In [76]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])
#             if tokens_only:
#                 yield gensim.utils.simple_preprocess(line)
#             else:
#                 # For training data, add tags
#                 yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [77]:
corpus = list(read_corpus("Records_All.txt"))
corpus_labels = list(read_labels("Records_All_Labels.txt"))
r = random.random()
random.shuffle(corpus, lambda: r)
random.shuffle(corpus_labels, lambda: r)

In [97]:
train_corpus = corpus[:736]
test_corpus = corpus[736:]
train_nums = corpus_labels[:736]
test_nums = corpus_labels[736:]

In [98]:
model = gensim.models.doc2vec.Doc2Vec(size=100, min_count=2, iter=55)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=100)

CPU times: user 2min 36s, sys: 2.61 s, total: 2min 38s
Wall time: 1min 6s


49142891

In [93]:
train_vecs = list(model.docvecs)
test_vecs = []
for elem in test_corpus:
    a = list(model.infer_vector(elem.words))
    test_vecs.append(a)
print(len(train_vecs))
print(len(corpus_labels))

1102
1104


In [88]:
import xgboost as xgb
import scipy.sparse
import numpy as np

In [89]:
xg_train = xgb.DMatrix(train_vecs, label=corpus_labels)
xg_test = xgb.DMatrix(test_vecs, label=test_nums)

In [90]:
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['max_depth'] = 10
param['silent'] = 0
param['num_class'] = 4

In [91]:
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 200
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
#error_rate = np.sum(pred != test_labels) / test_labels.shape[0]
#print('Test error using softmax = {}'.format(error_rate))

# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bst.predict(xg_test)#.reshape(np.array(test_labels).shape[0], 6)
pred_label = np.argmax(pred_prob, axis=1)
#error_rate = np.sum(pred != test_labels) / np.array(test_labels).shape[0]
#print('Test error using softprob = {}'.format(error_rate))
print(pred_label)
plot_confusion_matrix(confusion_matrix(test_nums,pred_label),classes=[1,2,3,4],normalize=True)

XGBoostError: b'[16:06:55] src/objective/multiclass_obj.cc:43: Check failed: preds.size() == (static_cast<size_t>(param_.num_class) * info.labels.size()) SoftmaxMultiClassObj: label size and pred size does not match\n\nStack trace returned 7 entries:\n[bt] (0) 0   libxgboost.so                       0x0000000116448e08 _ZN4dmlc15LogMessageFatalD2Ev + 40\n[bt] (1) 1   libxgboost.so                       0x0000000116446f79 _ZN4dmlc15LogMessageFatalD1Ev + 9\n[bt] (2) 2   libxgboost.so                       0x000000011649cef1 _ZN7xgboost3obj20SoftmaxMultiClassObj11GetGradientERKNSt3__16vectorIfNS2_9allocatorIfEEEERKNS_8MetaInfoEiPNS3_INS_9bst_gpairENS4_ISC_EEEE + 2257\n[bt] (3) 3   libxgboost.so                       0x0000000116445b4c _ZN7xgboost11LearnerImpl13UpdateOneIterEiPNS_7DMatrixE + 348\n[bt] (4) 4   libxgboost.so                       0x000000011645ad5a XGBoosterUpdateOneIter + 106\n[bt] (5) 5   _ctypes.cpython-36m-darwin.so       0x00000001090002c7 ffi_call_unix64 + 79\n[bt] (6) 